In [1]:
from datetime import datetime, timedelta
import pickle
from time import sleep

from bs4 import BeautifulSoup
from requests_html import HTMLSession
import yfinance as yf

In [2]:
EXCHANGE_PATHS = {
    'nasdaq': 'nasdaq',
    'nyse': 'newyorkstockexchange',
    'amex': 'americanstockexchange'}
base_url = 'https://www.advfn.com'
session = HTMLSession()

In [3]:
def get_soup(url):
    try:
        res = session.get(url)
        html = res.html.html
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    except BaseException as e:
        print(f'Problem retrieving page: {url}\n{e}')

In [4]:
def get_table(soup):
    try:
        table = soup.find('table', class_='market')
        return table
    except BaseExcetpion as e:
        print('Unable to obtain table')
        print(e)

In [5]:
def get_rows(table):
    try:
        rows = table.find_all('tr')
        rows = [
            row for row in rows if 'class' in row.attrs 
            and row.attrs['class'][0].startswith('ts')]
        return rows
    except BaseException as e:
        print('Unable to extract rows')
        print(e)

In [6]:
def is_valid(symbol):
    MAXLEN = 4
    return len(symbol) <= MAXLEN and symbol.isalpha()

In [7]:
def get_symbol(row):
    SYMBOL_COL_IDX = 1
    try:
        symbol = row.find_all('a')[SYMBOL_COL_IDX].text
        if is_valid(symbol):
            return symbol
    except BaseException as e:
        print('Unable to get symbol from row:', row)
        print(e)

In [8]:
def extract_symbols(soup):
    symbols = []
    table = get_table(soup)
    rows = get_rows(table)
    for row in rows:
        symbol = get_symbol(row)
        if symbol is not None:
            symbols.append(symbol)
    return symbols

In [9]:
def extract_all_symbols():
    all_symbols = []
    for exchange, path in EXCHANGE_PATHS.items():
        print(f'Beggining {exchange}\nLetter:')
        for letter in list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
            print(letter, end='')
            url = f'{base_url}/{exchange}/{path}.asp?companies={letter}'
            soup = get_soup(url)
            symbols = extract_symbols(soup)
            all_symbols += symbols
        print()
    return sorted(all_symbols)

In [10]:
all_symbols = extract_all_symbols()

Beggining nasdaq
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ
Beggining nyse
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ
Beggining amex
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [11]:
all_symbols = sorted(list(set(all_symbols)))

In [12]:
all_symbols[:10]

['A', 'AA', 'AAAP', 'AABA', 'AABC', 'AAC', 'AACC', 'AACE', 'AACI', 'AACQ']

In [13]:
all_symbols[-10:]

['ZWRK', 'ZWS', 'ZY', 'ZYME', 'ZYNE', 'ZYXI', 'ZZK', 'ZZO', 'ZZX', 'ZZZ']

In [14]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(all_symbols, f)

In [15]:
n = len(all_symbols)
n

11731

In [16]:
END = datetime.now()
START = END - timedelta(7)
test = yf.download(['AAPL', 'TSLA', 'ZQZZ'], start=str(START.date()), end=str(END.date()))['Adj Close']
test

[*********************100%%**********************]  3 of 3 completed

1 Failed download:
['ZQZZ']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Ticker,AAPL,TSLA,ZQZZ
Date,,,
2024-08-23,226.839996,220.320007,NaN
2024-08-26,227.179993,213.210007,NaN
2024-08-27,228.029999,209.210007,NaN
2024-08-28,226.490005,205.750000,NaN
2024-08-29,229.789993,206.279999,NaN


In [17]:
test2 = test.loc[:, test.iloc[-1, :].notnull()]
test2

Ticker,AAPL,TSLA
Date,,
2024-08-23,226.839996,220.320007
2024-08-26,227.179993,213.210007
2024-08-27,228.029999,209.210007
2024-08-28,226.490005,205.750000
2024-08-29,229.789993,206.279999


In [18]:
test2.columns

Index(['AAPL', 'TSLA'], dtype='object', name='Ticker')

In [19]:
start = str(START.date())
end = str(END.date())
start, end

('2024-08-23', '2024-08-30')

### Next Time
Code below is inefficient, bc it retries symbols if < $2 instead of removing them. 
Add to `omit` instead and do not rerun. (Update for all code chunks below.)

In [42]:
i = 0
active = []
failed = []
omit = []

while i <= n:
    first = i
    last = min(i + 100, n)
    batch = all_symbols[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    df = df.loc[:, df.iloc[-1, :].notnull()]
    batch_active = df.columns
    # Exclude if current val < 2.00
    df = df.loc[:, df.iloc[-1, :] >= 2.]
    batch_omit = [sym for sym in batch_active if sym not in df.columns]
    omit += list(batch_omit)
    active += list(df.columns)
    fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
    failed += fails
    sleep(3)
    i += 100

[                       0%%                      ]

$ABVA: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$AAAP: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$AANB: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[****                   9%%                      ]  9 of 100 completed

$ABGX: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[************          24%%                      ]  24 of 100 completed

...

[**********************88%%****************      ]  46 of 52 completed

$MTCT: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MSSN: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[**********************96%%********************  ]  50 of 52 completed

$MSSR: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[*********************100%%**********************]  52 of 52 completed

27 Failed downloads:
['MSBK', 'MSCC', 'MSW', 'MTDW', 'MSPD', 'MSLV', 'MSNQ', 'MSPX', 'MTDB', 'MSFG', 'MSHL', 'MTCT', 'MSSN', 'MSSR']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)')
['MSPR', 'MSII']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['MSCS', 'MTCN', 'MSG', 'MSP', 'MTAC', 'MSGN', 'MSON', 'MSL', 'MTCR', 'MTEC', 'MTBC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


In [43]:
'ACAI' in active, 'ACAI' in failed, 'ACAI' in omit 

(False, False, True)

### 2nd Attempt

In [44]:
len(active), len(omit)

(2871, 612)

In [45]:
n = len(failed)
n

3469

In [46]:
failed_again = []
i = 0
while i <= n:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100

[*                      2%%                      ]  2 of 100 completed

$AAAP: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$AANB: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ADIN: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ABVA: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[****                   9%%                      ]  9 of 100 completed

$ADK: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ACEC: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ACTS: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[*****                 11%%                      ]  11 of 100 completed

...

$MOLG: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)$MONE: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)

$MSSN: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MRAE: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[**********************94%%*******************   ]  65 of 69 completed

$MORF: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MSSR: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[*********************100%%**********************]  69 of 69 completed

69 Failed downloads:
['MSPR', 'MSII', 'MOTR']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['MSBK', 'MSW', 'MTDW', 'MSCC', 'MSPD', 'MOGN', 'MPWG', 'MOLX', 'MONM', 'MOSX', 'MRBA', 'MSLV', 'MROE', 'MRQQ', 'MRVC', 'MSNQ', 'MRNN', 'MSPX', 'MTDB', 'MRNT', 'MOSS', 'MRD', 'MSFG', 'MSHL', 'MRSS', 'MOVI', 'MTCT', 'MROI', 'MONE', 'MOLG', 'MSSN', 'MRAE', 'MORF', 'MSSR']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)')
['MPO', 'MOSC', 'MRLN', 'MON', 'MSAC', 'MPVD', 'MSCS', 'MOLC', 'MPAC', 'MOXC', 'MPRA', 'MSG', 'MTCN', 'MSP', 'MTAC', 'MRAC', 'MOTN', 'MSBF', 'MSON', 'MSGN', 'MOTV', 'MRTX', 'MSL', 'MPCT', 'MTCR', 'MOTA', 'MR', 'MRIC', 'MTEC', 'MOSY', 'MTBC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['MODN']: YFChartError('%ticker%: No data found, symbol may be delisted')


(0, 69)


### 3rd Attempt

In [47]:
'ACAI' in active, 'ACAI' in failed, 'ACAI' in omit 

(False, False, True)

In [48]:
len(active), len(omit)

(2873, 613)

In [49]:
failed = failed_again[:]
n = len(failed)
n

3466

In [50]:
failed_again = []
i = 0
while i <= n :
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100

[                       0%%                      ]

$ADIN: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ABVA: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$AANB: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$AAAP: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[****                   8%%                      ]  8 of 100 completed

$ACTS: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ADK: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$ACEC: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[*****                 11%%                      ]  11 of 100 completed

...

$MRSS: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MROI: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MONE: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MOLG: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[**********************91%%******************    ]  60 of 66 completed

$MSSN: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MORF: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
$MRAE: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)


[*********************100%%**********************]  66 of 66 completed

66 Failed downloads:
['MSPR', 'MSII', 'MOTR']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['MSBK', 'MSW', 'MTDW', 'MSCC', 'MPWG', 'MSPD', 'MSLV', 'MOLX', 'MONM', 'MOSX', 'MRBA', 'MROE', 'MRQQ', 'MRNN', 'MRVC', 'MSNQ', 'MSPX', 'MTDB', 'MOSS', 'MRNT', 'MSHL', 'MSFG', 'MRD', 'MTCT', 'MOVI', 'MRSS', 'MROI', 'MONE', 'MOLG', 'MSSN', 'MORF', 'MRAE', 'MSSR']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)')
['MPO', 'MOSC', 'MPVD', 'MSAC', 'MRLN', 'MON', 'MPRA', 'MTCN', 'MPAC', 'MOXC', 'MSCS', 'MSG', 'MSP', 'MSON', 'MTAC', 'MOTN', 'MRAC', 'MSBF', 'MOTV', 'MRTX', 'MSGN', 'MPCT', 'MSL', 'MTCR', 'MR', 'MTEC', 'MOTA', 'MRIC', 'MTBC', 'MOSY']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


$MSSR: possibly delisted; No price data found  (1d 2024-08-23 -> 2024-08-30)
(0, 66)


### 4th Attempt

In [51]:
len(active), len(omit)

(2873, 613)

In [52]:
failed = failed_again[:]
n = len(failed)
n

3466

In [53]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(active, f)

In [57]:
'INTI' in active, 'INTI' in omit

(False, True)

In [55]:
!say complete

In [ ]:
# Run...?
'''
failed_again = []
i = 0
while i <= n - 100:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100
'''